# Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim  # This is the critical import
from PIL import Image      # Should show 9.x.x or later
from torchsummary import summary
# import os
# import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


In [12]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_convs):
        super().__init__()
        self.convs = nn.Sequential()
        
        # Membuat layer konvolusi sesuai jumlah yang ditentukan
        for i in range(num_convs):
            input_channels = in_channels if i == 0 else out_channels
            self.convs.add_module(f'conv{i+1}', nn.Conv2d(
                input_channels, out_channels, kernel_size=3, padding=1))
            self.convs.add_module(f'bn{i+1}', nn.BatchNorm2d(out_channels))
            self.convs.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Skip connection dengan 1x1 conv untuk menyesuaikan dimensi
        self.skip = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        identity = self.skip(x)  # Proses skip connection
        out = self.convs(x)      # Jalur utama melalui konvolusi
        out = self.pool(out)     # Pooling setelah konvolusi
        out += identity          # Tambahkan skip connection
        return F.relu(out)       # Aktivasi akhir

class ResidualVGG16(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        
        # Membangun blok-blok residual sesuai arsitektur VGG16
        self.blocks = nn.Sequential(
            ResidualBlock(3, 32, 2),       # Blok 1: 2 konvolusi 64 channel
            ResidualBlock(32, 64, 2),     # Blok 2: 2 konvolusi 128 channel
            ResidualBlock(64, 128, 3),    # Blok 3: 3 konvolusi 256 channel
            ResidualBlock(128, 256, 3),    # Blok 4: 3 konvolusi 512 channel
            ResidualBlock(256, 256, 3),    # Blok 5: 3 konvolusi 512 channel
        )
        
        # Classifier dengan fully-connected layers
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.blocks(x)               # Loloskan melalui semua blok residual
        x = torch.flatten(x, 1)          # Flatten feature maps
        x = self.classifier(x)           # Loloskan melalui classifier
        return x



In [13]:

model = ResidualVGG16()  # Your model
summary(model, input_size=(3, 224, 224))  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             128
       BatchNorm2d-2         [-1, 32, 112, 112]              64
            Conv2d-3         [-1, 32, 224, 224]             896
       BatchNorm2d-4         [-1, 32, 224, 224]              64
              ReLU-5         [-1, 32, 224, 224]               0
            Conv2d-6         [-1, 32, 224, 224]           9,248
       BatchNorm2d-7         [-1, 32, 224, 224]              64
              ReLU-8         [-1, 32, 224, 224]               0
         MaxPool2d-9         [-1, 32, 112, 112]               0
    ResidualBlock-10         [-1, 32, 112, 112]               0
           Conv2d-11           [-1, 64, 56, 56]           2,112
      BatchNorm2d-12           [-1, 64, 56, 56]             128
           Conv2d-13         [-1, 64, 112, 112]          18,496
      BatchNorm2d-14         [-1, 64, 1

In [4]:
# Contoh penggunaan
# if __name__ == "__main__":
#     model = ResidualVGG16(num_classes=2)
#     input_tensor = torch.randn(1, 3, 224, 224)  # Contoh input
#     output = model(input_tensor)
#     print(f"Output shape: {output.shape}")   

# Load Data

In [5]:
# tumor_dir_path = "./Dataset/processed/yes"
# no_tumor_dir_path = "./Dataset/processed/no"

# tumor_image_paths =  os.listdir(tumor_dir_path)
# tumor_image_paths = [os.path.join(tumor_dir_path, path) for path in tumor_image_paths]

# no_tumor_image_paths =  os.listdir(no_tumor_dir_path)
# no_tumor_image_paths = [os.path.join(no_tumor_dir_path, path) for path in no_tumor_image_paths]
# no_tumor_image_paths

# display(tumor_image_paths, no_tumor_image_paths)

In [6]:
dataset_dir_path = '/kaggle/input/brain-tumor-dataset'

In [7]:
# Update the data preparation function
def prepare_data(data_dir='your_dataset'):
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], 
        #                      std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                      std=[0.229, 0.224, 0.225])
    ])

    # Load datasets
    train_dataset = datasets.ImageFolder(
        root=f'{data_dir}/train',
        transform=train_transform
    )
    
    val_dataset = datasets.ImageFolder(
        root=f'{data_dir}/val',
        transform=val_transform
    )

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
    
    return train_loader, val_loader

In [8]:
train_loader, val_loader = prepare_data(data_dir=dataset_dir_path)

In [ ]:
# Hyperparameters
NUM_EPOCHS = 20


# Initialize model, loss, optimizer
model = ResidualVGG16(num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images, labels
        
        print(f'Start batch-{batch_idx}')
        optimizer.zero_grad()
        outputs = model(images)

        # Forward pass
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images, labels
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], '
              f'Val Accuracy: {100 * correct / total:.2f}%')

Start batch-0
Start batch-1
Start batch-2
Start batch-3
Start batch-4
Start batch-5
Start batch-6
Start batch-7
Start batch-8
Start batch-9
Start batch-10
Start batch-11
Start batch-12
Start batch-13
Start batch-14
Start batch-15
Start batch-16
Start batch-17
Start batch-18
Start batch-19
Start batch-20
Start batch-21
Start batch-22
Start batch-23
Start batch-24
Start batch-25
Start batch-26
Start batch-27
Start batch-28
Start batch-29
Start batch-30
Start batch-31
Start batch-32
Start batch-33
Start batch-34
Start batch-35
Start batch-36
Start batch-37
Start batch-38
Start batch-39
Start batch-40
Start batch-41
Start batch-42
Start batch-43
Start batch-44
Start batch-45
Start batch-46
Start batch-47
Start batch-48
Start batch-49
Start batch-50
Start batch-51
Start batch-52
Start batch-53
Start batch-54
Start batch-55
Start batch-56
Start batch-57
Start batch-58
Start batch-59
Start batch-60
Start batch-61
Start batch-62
Start batch-63
Start batch-64
Start batch-65
Start batch-66
Start